# Bronze Layer Tables
## orders_raw

If you have cleared the workspace. run the Setup command and then comment the cell for proper function

In [0]:
-- %run ./00_Setup

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE orders_raw
COMMENT "Raw orders data"
AS SELECT 
  order_id, 
  order_timestamp, 
  customer_id, 
  quantity,
  books
FROM cloud_files(
  "/Volumes/workspace/strata_lab/entrenamiento/book_store/orders/", "parquet",
  map("schema", "order_id STRING, order_timestamp LONG, customer_id STRING, quantity LONG, books array<struct<book_id:string>>")
)

In [0]:
-- Creates the customers table from raw JSON data
CREATE OR REFRESH LIVE TABLE sales_customers
COMMENT "Sales by customer"
AS SELECT * FROM json.`/Volumes/workspace/strata_lab/entrenamiento/book_store/customers/`

In [0]:
-- Cleans orders data; enforces valid order_id and joins customers
CREATE OR REFRESH STREAMING LIVE TABLE orders_cleaned
  ( CONSTRAINT valid_order_number EXPECT(order_id IS NOT NULL) ON VIOLATION DROP ROW)
  COMMENT "Cleaned orders data with valid order_id" AS
SELECT
  o.order_id,
  o.quantity,
  o.customer_id,
  c.profile:first_name AS f_name,
  c.profile:last_name AS l_name,
  cast(from_unixtime(order_timestamp, 'yyyy-MM-dd HH:mm:ss') AS timestamp) AS order_timestamp,
  c.profile:address:country as country,
  o.books as books
FROM STREAM(LIVE.orders_raw) o
LEFT JOIN LIVE.sales_customers c
ON o.customer_id = c.customer_id

# Gold Tables

In [0]:
-- Aggregates daily book counts for each USA customer (gold layer)
CREATE OR REFRESH LIVE TABLE cn_daily_customer_books
COMMENT "Daily number of books per customer in USA" AS
SELECT
  customer_id,
  f_name,
  l_name,
  date_trunc("DD", order_timestamp) AS order_date,
  sum(quantity) AS books_counts
FROM LIVE.orders_cleaned
WHERE country = 'USA'
GROUP BY customer_id, f_name, l_name, date_trunc("DD", order_timestamp)